In [4]:
%%file finetune.py
# Fine tune.
from constant import *
import os
import numpy as np
import time
import cPickle
from pylab import *
import argparse
#%matplotlib inline

parser = argparse.ArgumentParser()
parser.add_argument('--gpu', default=0, required=False)
parser.add_argument('--net_name', required=True)
parser.add_argument('--type_str', required=True) # crop / nocrop / aperture
args = parser.parse_args()
gpu = int(args.gpu)
net_name = args.net_name
type_str = args.type_str

import caffe

plt.rcParams['figure.figsize'] = (20, 20)

niter = 10000
# Losses will also be stored in the log.
train_loss = np.zeros(niter)
train_accuracy = np.zeros(niter)
test_accuracy = {}

caffe.set_device(gpu)
caffe.set_mode_gpu()

print 'Processing: finetune_alexnet_{}_{}'.format(type_str, net_name), 'on GPU', gpu, ',', type_str

solver = caffe.SGDSolver(result_root + 'model/finetune_alexnet_{}_{}/solver.prototxt'.format(type_str, net_name))
solver.net.copy_from(imagenet_root + 'model/bvlc_alexnet/bvlc_alexnet.caffemodel')

start_time = time.time()
# We run the solver for niter times, and record the training loss.
for it in range(niter):
    solver.step(1)  # SGD by Caffe
    train_loss[it] = solver.net.blobs['loss'].data
    train_accuracy[it] = solver.net.blobs['accuracy_train'].data
    second = int(time.time() - start_time)
    estimated = int(float(niter) / (it + 1) * second)
    now_time = time.strftime("%H:%M:%S", time.gmtime(second))
    estimated_time = time.strftime("%H:%M:%S", time.gmtime(estimated))
    if it % 100 == 0 or (it <= 1000 and it % 10 == 0):
        test_iters = 10
        accuracy = 0
        for i in arange(test_iters):
            solver.test_nets[0].forward()
            accuracy += solver.test_nets[0].blobs['accuracy_test'].data
        accuracy /= test_iters
        test_accuracy[it] = accuracy
        print '[{} / {}] iter{:6} | train_loss={:10.6f}, train_accuracy={:10.6f}, test_accuracy={:10.6f}'.format(now_time, estimated_time, it, float(train_loss[it]), float(train_accuracy[it]), accuracy)
    elif it % 10 == 0:
        print '[{} / {}] iter{:6} | train_loss={:10.6f}, train_accuracy={:10.6f}'.format(now_time, estimated_time, it, float(train_loss[it]), float(train_accuracy[it]))
        
solver.net.save(result_root + 'model/finetune_alexnet_{}_{}/finetune_alexnet_{}_{}.caffemodel'.format(type_str, net_name, type_str, net_name))

with open(result_root + 'finetune/train_loss_{}_{}.pickle'.format(type_str, net_name), 'wb') as f:
    cPickle.dump(train_loss, f)
with open(result_root + 'finetune/train_accuracy_{}_{}.pickle'.format(type_str, net_name), 'wb') as f:
    cPickle.dump(train_accuracy, f)
with open(result_root + 'finetune/test_accuracy_{}_{}.pickle'.format(type_str, net_name), 'wb') as f:
    cPickle.dump(test_accuracy, f)

print 'done'

Overwriting finetune.py


In [2]:
# Create network files.
import os
from constant import *

type_str = 'crop_obj'
#net_names = ['0', '10', '20', '30', '40', '50', '60', '70', '80', '90', '100' 'all']
net_names = ['0', '20', '40', '60', '80', '100', 'all']

with open(result_root + 'model/finetune_alexnet_prototype/deploy.prototxt') as f:
    deploy_lines = f.readlines()
with open(result_root + 'model/finetune_alexnet_prototype/solver.prototxt') as f:
    solver_lines = f.readlines()
with open(result_root + 'model/finetune_alexnet_prototype/train_val.prototxt') as f:
    train_val_lines = f.readlines()


for net_name in net_names:
    folder = '{}model/finetune_alexnet_{}_{}/'.format(result_root, type_str, net_name)
    if not os.path.exists(folder):
        os.makedirs(folder)
    solver_lines[0] = 'net: "{}model/finetune_alexnet_{}_{}/train_val.prototxt"\n'.format(result_root, type_str, net_name)
    solver_lines[14] = 'snapshot_prefix: "{}model/finetune_alexnet_{}_{}/snapshot"\n'.format(result_root, type_str, net_name)
    train_val_lines[15] = '    source: "{}dataset/train_{}_{}.txt"\n'.format(imagenet_root, type_str, net_name)
    train_val_lines[36] = '    source: "{}dataset/test_{}_{}.txt"\n'.format(imagenet_root, type_str, net_name)
    with open(folder + 'deploy.prototxt', 'w') as f:
        f.writelines(deploy_lines)
    with open(folder + 'solver.prototxt', 'w') as f:
        f.writelines(solver_lines)
    with open(folder + 'train_val.prototxt', 'w') as f:
        f.writelines(train_val_lines)
    